<a href="https://colab.research.google.com/github/sundarjhu/UACJ_Jornada2021/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
  from astroquery.sdss import SDSS
except:
  !pip install astroquery
  from astroquery.sdss import SDSS
from astropy import coordinates as coords
#pos = coords.SkyCoord('202.45748d 29.579716d', frame = 'icrs')
pos = coords.SkyCoord.from_name("KISSR 298")
xid = SDSS.query_region(pos, spectro = True)
sp = SDSS.get_spectra(matches = xid)

     |████████████████████████████████| 6.5MB 6.9MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 3.2MB 51.5MB/s 
  Created wheel for astroquery: filename=astroquery-0.4.1-cp37-none-any.whl size=3831874 sha256=1a13a5084133703b037a58100aa6af85f2aefe67f38b3bfea2732ff34f399dfa
  Stored in directory: /root/.cache/pip/wheels/88/f8/b7/a254cd96e808f708bc0b7d755a8e095c56fbbe94099d7b464f
Successfully built astroquery


/usr/local/lib/python3.7/dist-packages/astroquery/sdss/core.py:865: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  comments='#'))


In [11]:
sp[0][1].data

FITS_rec([(13.479021, 3.5816, 0.06797916, 0, 0, 1.1088611 , 9.002381, 13.217207),
          (19.215475, 3.5817, 0.06977836, 0, 0, 1.108475  , 9.212716, 13.737914),
          (14.267138, 3.5818, 0.07592517, 0, 0, 1.108091  , 9.300478, 13.168895),
          ...,
          ( 8.162261, 3.9645, 0.35342726, 0, 0, 0.7729795 , 4.512267, 12.689797),
          (11.752623, 3.9646, 0.        , 0, 0, 0.77300555, 4.811207, 12.891481),
          (15.343176, 3.9647, 0.28280747, 0, 0, 0.77303153, 4.932665, 12.677738)],
         dtype=(numpy.record, [('flux', '>f4'), ('loglam', '>f4'), ('ivar', '>f4'), ('and_mask', '>i4'), ('or_mask', '>i4'), ('wdisp', '>f4'), ('sky', '>f4'), ('model', '>f4')]))

In [8]:
len(sp[0])

10